In [1]:
%reload_ext watermark
%reload_ext autoreload
%autoreload 2
%watermark -v -p numpy,sklearn,pandas
%watermark -v -p cv2,PIL,matplotlib
%watermark -v -p torch,torchvision,torchaudio
%watermark -v -p tensorflow,tensorboard
%matplotlib inline
%config InlineBackend.figure_format='retina'
%config IPCompleter.use_jedi = False

from IPython.display import display, Markdown, HTML, Javascript
display(HTML('<style>.container { width:%d%% !important; }</style>' % 80))

import sys, os, io, time, random, math
import json, base64, requests
import os.path as osp
import numpy as np

def _IMPORT_(x):
    try:
        segs = x.split(' ')
        g = globals()
        if 'github.com' in segs[1]:
            uri = segs[1].replace('github.com', 'raw.githubusercontent.com')
            mod = uri.split('/')
            for s in ['main', 'master']:
                uri = 'https://' + '/'.join(mod[:-1]) + '/main/' + mod[-1] + '.py'
                x = requests.get(uri).text
                if x.status == 200:
                    break
        elif 'gitee.com' in segs[1]:
            mod = segs[1].split('/')
            for s in ['/raw/main/', '/raw/master/']:
                uri = 'https://' + '/'.join(mod[:3]) + s + '/'.join(mod[3:]) + '.py'
                x = requests.get(uri).text
                if x.status == 200:
                    break
        elif segs[1][0] == '/':
            with open(segs[1] + '.py') as fr:
                x = fr.read()
        exec(x, g)
    except:
        pass


CPython 3.6.9
IPython 7.16.1

numpy 1.19.5
sklearn 0.24.0
pandas 1.1.5
CPython 3.6.9
IPython 7.16.1

cv2 4.5.1
PIL 6.2.2
matplotlib 3.3.3
CPython 3.6.9
IPython 7.16.1

torch 1.8.0.dev20210103+cu101
torchvision 0.9.0.dev20210103+cu101
torchaudio not installed
CPython 3.6.9
IPython 7.16.1

tensorflow 2.6.0
tensorboard 2.6.0


In [2]:

###
### Torch ###
###

_IMPORT_('import torch')
_IMPORT_('import torch.nn as nn')
_IMPORT_('import torch.nn.functional as F')
_IMPORT_('import torch.optim as O')
_IMPORT_('from torchvision import models as M')
_IMPORT_('from torchvision import transforms as T')
_IMPORT_('from torch.utils.data import Dataset, DataLoader')


In [3]:
from torchvision import datasets
from PIL import Image

## Pytorch Demo

In [4]:
class TorchModel(nn.ModuleList):
    def __init__(self):
        super(TorchModel, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=(3, 3), padding=(1, 1))
        self.bn1 = nn.BatchNorm2d(32, eps=1e-5)
        self.relu1 = nn.ReLU()
        self.avgpoll = nn.AdaptiveAvgPool2d((1, 1))
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=10, kernel_size=(1, 1), padding=(0, 0))
        self.flatten = nn.Flatten()
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.avgpoll(x)
        x = self.conv2(x)
        return self.flatten(x)

In [12]:
train_dataset = datasets.MNIST(
    root='/data/nb_data/datasets/', train=True, download=False,
    transform=T.Compose([T.Lambda(lambda x: x.convert('RGB')), T.ToTensor(), T.Normalize((0.1307,), (0.3081,))]))
train_loader = DataLoader(train_dataset, batch_size=1024, shuffle=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TorchModel().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = O.Adam(model.parameters(), lr=1e-4)
# not learning, only for onnx
for epoch in range(1):
    for image, target in train_loader:
        image, target = image.to(device), target.to(device)
        output = model(image)
        loss = criterion(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [18]:
torch.onnx.export(
    model, torch.randn((1, 3, 28, 28)),
    '/data/nb_data/tmp/torch_model.onnx', 
    verbose=True) 

graph(%input.1 : Float(1, 3, 28, 28, strides=[2352, 784, 28, 1], requires_grad=0, device=cpu),
      %conv2.weight : Float(10, 32, 1, 1, strides=[32, 1, 1, 1], requires_grad=1, device=cpu),
      %conv2.bias : Float(10, strides=[1], requires_grad=1, device=cpu),
      %17 : Float(32, 3, 3, 3, strides=[27, 9, 3, 1], requires_grad=0, device=cpu),
      %18 : Float(32, strides=[1], requires_grad=0, device=cpu)):
  %16 : Float(1, 32, 28, 28, strides=[25088, 784, 28, 1], requires_grad=1, device=cpu) = onnx::Conv[dilations=[1, 1], group=1, kernel_shape=[3, 3], pads=[1, 1, 1, 1], strides=[1, 1]](%input.1, %17, %18)
  %12 : Float(1, 32, 28, 28, strides=[25088, 784, 28, 1], requires_grad=1, device=cpu) = onnx::Relu(%16) # /usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1111:0
  %13 : Float(1, 32, 1, 1, strides=[32, 1, 1, 1], requires_grad=1, device=cpu) = onnx::GlobalAveragePool(%12) # /usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:911:0
  %14 : Float(1, 10, 1, 1, s

## References

- [Creating and Modifying ONNX Model Using ONNX Python API][1]

[1]: https://leimao.github.io/blog/ONNX-Python-API/

In [ ]:
train_dataset[0]